In [1]:
%pylab nbagg
from tvb.simulator.lab import *
import os
import numpy
import networkx

Populating the interactive namespace from numpy and matplotlib
   INFO  log level set to INFO
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.wilson_cowan.WilsonCowan.state_variable_range = Final(field_type=<class 'dict'>, default={'E': array([0., 1.]), 'I': array([0., 1.])}, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.stefanescu_jirsa.ReducedSetFitzHughNagumo.state_variable_range = Final(field_type=<class 'dict'>, default={'xi': array([-4.,  4.]), 'eta': array([-3.,  3.]), 'alpha': array([-4.,  4.]), 'beta': array([-3.,  3.])}, required=True)
WARNING  default contains values out of the declared domain. Ex 1.0 
   attribute  tvb.simulator.models.stefanescu_jirsa.ReducedSetHindmarshRose.a = NArray(label=':math:`a`', dtype=float64, default=array([1.]), dim_names=(), ndim=None, required=True)
WARNING  def

WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.wong_wang_exc_io_inh_i.ReducedWongWangExcIOInhI.state_variable_range = Final(field_type=<class 'dict'>, default={'S_e': array([0., 1.]), 'S_i': array([0., 1.])}, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.linear.Linear.state_variable_range = Final(field_type=<class 'dict'>, default={'x': array([-1,  1])}, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.hopfield.Hopfield.state_variable_range = Final(field_type=<class 'dict'>, default={'x': array([-1.,  2.]), 'theta': array([0., 1.])}, required=True)
WARNING  Field seems mutable and has a default value. Consider using a lambda as a value factory 
   attribute tvb.simulator.models.epileptor.Epileptor.state_var

# Tutorial: Modeling The Impact of Structural Lesions -- Part I: Modeling Lesions

In this tutorial we will explore different lesion strategies as presented in [1]. 

Two main strategies were used:

  * sequential single deletion     
    * random
    * targeted    
  * focal

The first type, random method,  was aiming to provide a structural failure analysis, that is, to analyse systematically the robustness of the network. The first type, second method, provides an intemediate step toward specific lesioning, by taking into account graph metrics like degree, strength and betweeness centrality to select a target node.  

The second type was intended to evaluate functional failure analysis. In this strategy we not only specify a focal node that belongs to a region of the Default Mode Network (DMN) [2, 3] but nodes within a localized extent are deleted as well. 
The size of the lesions represent 5% of the cortical surface.



Here, we will explore what these strategies are about and compute + store some graph metrics of the lesioned connectivities. New matrices won't be saved. The first thing to do is creating an instance of a Connectivity datatype with the connectome we'll use as a control ('healthy').  

There are several connectivity matrices available, one of them is the 998 regions originally used in [3]

In [2]:
white_matter = connectivity.Connectivity.from_file("connectivity_998.zip")
white_matter.configure()
nor = white_matter.number_of_regions

WARNING  File 'average_orientations' not found in ZIP.
WARNING  File 'cortical' not found in ZIP.
WARNING  File 'hemispheres' not found in ZIP.


Let's have a look at the properties of this matrix

In [3]:
white_matter

,value
Number of connections,35730
Number of regions,998
Undirected,False
"areas [min, median, max]","[1.5, 1.5, 1.5]"
areas dtype,float64
areas shape,"(998,)"
"tract_lengths (connections) [min, median, max]","[4.25, 27.0265, 189.5]"
"tract_lengths [min, median, max]","[0, 0, 189.5]"
"tract_lengths-non-zero [min, median, max]","[4.25, 27.0265, 189.5]"
"weights [min, median, max]","[0, 0, 0.914943]"


#### Graph Metrics
Graph metrics computed after each node deletion are node degree, strength, global efficiency, largest component and betweeness centrality. 

We'll import from the `tvb.analyzers.graph` a few useful functions:

In [4]:
from tvb.analyzers.graph import betweenness_bin, distance_inv, efficiency_bin, get_components_sizes, sequential_random_deletion

### Sequential random deletion 


In orider to be consistent about the lesions, we'll first set the Numpy's random number genrator seed -- this will guarantee that you'll get the same random sequence.

A small example is given below:

In [5]:
my_seed = 42
my_random_state = numpy.random.RandomState(my_seed)

Next, a sequence of random sequence of integer numbers drawn from the regions indices, without replacement, will define the order in which the nodes are going to be deleted.

In [6]:
random_sequence = my_random_state.choice(numpy.arange(nor), nor-2, replace=False)
ns, nd, ge, lc = sequential_random_deletion(white_matter, random_sequence, nor)

In [1] the graph metrics were computed for 25 different random sequences. So, the next cell contains a loop to do this. It is most certainly not the way to do it since it takes a long long time. This task can/should be parallelized without a problem.

In [7]:
my_seeds = [0, 1, 2, 3, 5, 7, 12, 13, 19, 21, 27, 33, 42, 53, 64, 67, 73, 77, 81, 84, 86, 89, 92, 97, 99]

In [10]:
import os
if not os.path.exists('data'):
    os.makedirs('data')
for this_seed in my_seeds:
    my_random_state = numpy.random.RandomState(this_seed)
    random_sequence = my_random_state.choice(numpy.arange(nor), nor-2, replace=False)
    ns, nd, ge, lc = sequential_random_deletion(white_matter, random_sequence, nor)
    # save them
    numpy.save('data/node_strength_' + str(this_seed) + '.npy' , ns)
    numpy.save('data/node_degree_' + str(this_seed) + '.npy' , nd)
    numpy.save('data/global_efficiency_' + str(this_seed) + '.npy' , ge)
    numpy.save('data/largest_component_' + str(this_seed) + '.npy' , lc)    

KeyboardInterrupt: 

### Sequential targeted deletions

In [ ]:
ns, nd, nbc, ge, lc = sequential_targeted_deletion(white_matter, nor)

In [ ]:
# save them
numpy.save('data/node_strength_' + 'ns' + '.npy' , ns)
numpy.save('data/node_degree_'   + 'nd' + '.npy' , nd)
numpy.save('data/node_betweenness_centrality_'    + 'nbc' + '.npy' , nbc)
numpy.save('data/global_efficiency_' + 'ns_nd_bc' + '.npy' , ge)
numpy.save('data/largest_component_' + 'ns_nd_bc' + '.npy' , lc)

Now, we'll load the metrics computed for **sequential random deletion**...

In [ ]:
global_efficiency = numpy.zeros((996, len(my_seeds)))
largest_component = numpy.zeros((996, len(my_seeds)))
for i, this_seed in enumerate(my_seeds):
    # load them
    global_efficiency[:, i] = numpy.load('data/global_efficiency_' + str(this_seed) + '.npy')
    largest_component[:, i] = numpy.load('data/largest_component_' + str(this_seed) + '.npy')

And the ones computed for **sequential targeted deletion** ...

In [ ]:
ge = numpy.load('data/global_efficiency_' + 'ns_nd_bc' + '.npy' )
lc = numpy.load('data/largest_component_' + 'ns_nd_bc' + '.npy' )

#### Global  efficiency

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,8))
ax.plot(global_efficiency.mean(axis=1), 'k', linewidth=4, label='random')
ax.plot(ge[:, 0], 'g', linewidth=4, label='targeted (strength)')
ax.plot(ge[:, 1], 'b', linewidth=4, label='targeted (degree)')
ax.plot(ge[:, 2], 'r', linewidth=4, label='targeted (centrality)')

plt.xlabel('Number of Deleted Nodes', fontsize=24)
plt.ylabel('Global Efficiency (binary)', fontsize=24)

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(24) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(24) 
    
plt.legend()

#### Size of the largest component

In [ ]:
fig, ax = plt.subplots(1, figsize=(20,8))
ax.plot(largest_component.mean(axis=1), 'k', linewidth=4, label='random')
ax.plot(lc[:, 0], 'g', linewidth=4, label='targeted (strength)')
ax.plot(lc[:, 1], 'b', linewidth=4, label='targeted (degree)')
ax.plot(lc[:, 2], 'r', linewidth=4, label='targeted (centrality)')

plt.xlabel('Number of Deleted Nodes',   fontsize=24)
plt.ylabel('Size of Largest Component', fontsize=24)
for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(24) 
for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(24) 
    
plt.legend()

### Localized deletions

Localized deletions represent area removal, that is, a certain number of nodes and their connections around a focal point were deleted. This strategy was aiming at dynamic and functional failure analysis, since more evident changes in the dynamic patters are expected. 

The central locations were abribitrarily chosen. In this example we'll only remove node 194 [index 193] and its 49 nearest neighbours (5% of the represented cortical surface). Despite having the fibre distances, the nearest neighbouts were etermined by the Euclidean distance. 


In [ ]:
def localized_deletions(white_matter, lesion_indices, start, end, k=49):
    """
    
    Remove a focal node and its k nearest neighbours.
    Lesions are constrained to be intrahemispheric.
    
    
    Parameters
    ----------
    
    white_matter   : a tvb Connectivity datatype    
    lesion_indices : array
                     an index vector with the focal nodes that will be deleted.
                     
    k              : int 
                     neighbours to be deleted.
                                        
    start          : int 
                     first index of a given hemisphere
                     
    end            : int 
                     last index of a hemisphere
                     
                     
    Notes
    -----
    eg, first hemipshere start=0; end=nor//2
        second hemisphere: start=nor//2; end=nor
        
        and nor is white_matter.number_of_regions
        
    .. author:: Paula Sanz Leon
    """

    for lesion in lesion_indices:

        # compute distances and such
        xo = white_matter.centres[lesion, 0]
        yo = white_matter.centres[lesion, 1]
        zo = white_matter.centres[lesion, 2]

        # only compute distance wrt to intra-hemispheric neighbours
        nor = white_matter.number_of_regions
        distances = numpy.sqrt((white_matter.centres[start:end, 0] - xo)**2 
                    +  (white_matter.centres[start:end, 1] - yo)**2 
                    +  (white_matter.centres[start:end, 2] - zo)**2)

        # get neighbouring nodes
        sorted_euclidean = numpy.argsort(distances)
        good_indices     = sorted_euclidean[:k+1]

        # lesion
        new_connectivity_weights = white_matter.weights.copy()
        new_connectivity_weights[good_indices, :] = 0.0
        new_connectivity_weights[:, good_indices] = 0.0
        
        return new_connectivity_weights, good_indices

In [ ]:
lesion_indices           = (numpy.array([194]) - 1).astype(int)
lesioned_matrix_weights, idxs  = localized_deletions(white_matter, lesion_indices, 0, nor//2)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

fig, ax = plt.subplots(1, figsize=(25,20))

# for visualization purposes
data = lesioned_matrix_weights.copy()
# binarize
data[data > 0] = 1
data[idxs, :] = 3
data[:, idxs] = 3
data[lesion_indices, :] = 2
data[:, lesion_indices] = 2


# make a color map of fixed colors
cmap = colors.ListedColormap(['black', 'white', 'red', 'blue'])
bounds=[0,1,2,3,4]
norm = colors.BoundaryNorm(bounds, cmap.N)

p = ax.pcolormesh(data, cmap=cmap, norm=norm)
ax.invert_yaxis()
cbar = fig.colorbar(p, cmap=cmap, norm=norm, boundaries=bounds, ticks=[0.5, 1.5, 2.5, 3.5])
cbar.ax.set_yticklabels(['no connections', 'connections', 'lesion site', 'neighbours'], fontsize=24)

xticks = range(998)
yticks = range(998)

xticklabels=['L194', ]
yticklabels=['L194', ]

ax.set_xticks(xticks[193:194])
ax.set_xticklabels(xticklabels)

ax.set_yticks(yticks[193:194])
ax.set_yticklabels(yticklabels)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(24) 

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(24) 




In [ ]:
data = white_matter.weights.copy()

from matplotlib import colors
fig, ax = plt.subplots(1, figsize=(25,20))

# for visualization purposes
data[data > 0] = 1


# make a color map of fixed colors
cmap = colors.ListedColormap(['black', 'white'])
bounds=[0,1,2]
norm = colors.BoundaryNorm(bounds, cmap.N)

p = ax.pcolormesh(data, cmap=cmap, norm=norm)
ax.invert_yaxis()
cbar = fig.colorbar(p, cmap=cmap, norm=norm, boundaries=bounds, ticks=[0.5, 1.5])
cbar.ax.set_yticklabels(['no connections', 'connections', 'lesion site', 'neighbours'], fontsize=24)

xticks = range(998)
yticks = range(998)

xticklabels=['RH']
yticklabels=['RH']

ax.set_xticks(xticks[998//4:998//4+1])
ax.set_xticklabels(xticklabels)

ax.set_yticks(yticks[998//4:998//4+1])
ax.set_yticklabels(yticklabels)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(24) 

for tick in ax.xaxis.get_major_ticks():
    tick.label.set_fontsize(24) 

### References

[1] Alstott et al. (2009) Modeling the impact of lesions in the human brain. Plos Comp Bio.

[2] Damoiseaux et al. (2006) Consistent resting-state networks across healthy subjects. PNAS

[3] Honey et al. (2009) Predicting human resting-state functional connectivity from structural connectivity. PNAS.